In [6]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from time import gmtime, strftime

role = "arn:aws:iam::342474125894:role/service-role/AmazonSageMaker-ExecutionRole-20190405T234154"
account_id = role.split(':')[4]
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
bucket = sagemaker_session.default_bucket()

print(account_id)
print(region)
print(role)
print(bucket)

342474125894
ap-southeast-1
arn:aws:iam::342474125894:role/service-role/AmazonSageMaker-ExecutionRole-20190405T234154
sagemaker-ap-southeast-1-342474125894


<h3> Model information

In [7]:
"""
# XGBoost container from sdk v2

import sagemaker

result = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region="ap-southeast-1",
    version="1.0-1",
    py_version="py3",
    instance_type="ml.m5.2xlarge",
    accelerator_type=None,
    image_scope="inference",
)

print(result)
"""
container = "121021644041.dkr.ecr.ap-southeast-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3"

model_file_name = "xgboost-model-"
model_name = model_file_name + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

# Model artifact from training job
key = "script-mode-container-xgb-2020-08-10-16-33-10-408/output/model.tar.gz"
model_url = 'https://s3-{}.amazonaws.com/{}/{}'.format(region,bucket,key)

<h3> Low level API (Use High level below) </h3>

In [25]:
sm_client = boto3.client('sagemaker')

print (model_url)

primary_container = {
    'Image': container,
    'ModelDataUrl': model_url,
}

create_model_response2 = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response2['ModelArn'])

https://s3-ap-southeast-1.amazonaws.com/sagemaker-ap-southeast-1-342474125894/script-mode-container-xgb-2020-08-10-16-33-10-408/output/model.tar.gz
arn:aws:sagemaker:ap-southeast-1:342474125894:model/xgboost-model-2020-08-10-17-07-48


In [26]:

endpoint_config_name = 'xgboost-model-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m4.xlarge',
        'InitialInstanceCount':1,
        'InitialVariantWeight':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

xgboost-model-2020-08-10-17-07-59
Endpoint Config Arn: arn:aws:sagemaker:ap-southeast-1:342474125894:endpoint-config/xgboost-model-2020-08-10-17-07-59


In [27]:
import time

endpoint_name = 'xgboost-model-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Status: " + status)

while status=='Creating':
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp['EndpointStatus']
    print("Status: " + status)

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

DEMO-XGBoostEndpoint-2020-08-10-17-08-06
arn:aws:sagemaker:ap-southeast-1:342474125894:endpoint/demo-xgboostendpoint-2020-08-10-17-08-06
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:ap-southeast-1:342474125894:endpoint/demo-xgboostendpoint-2020-08-10-17-08-06
Status: InService


<h3> High level

In [9]:
from sagemaker.model import Model

# Cant use the training custom container because it does not implement sagemaker-inference toolkit, hence switch back to xgboost prebuilt container image
model_data = "s3://sagemaker-ap-southeast-1-342474125894/script-mode-container-xgb-2020-08-10-16-33-10-408/output/model.tar.gz"
model = sagemaker.model.Model(model_data, container, role, sagemaker_session=sagemaker_session)

In [11]:
model.deploy(initial_instance_count=1, instance_type="ml.t2.large")

-------------!

<h3> Inference

In [2]:
import boto3

runtime_client = boto3.client('runtime.sagemaker')


payload = "1,2,3,4,5"
endpoint_name = 'DEMO-XGBoostEndpoint-2020-08-10-17-08-06'

response = runtime_client.invoke_endpoint(EndpointName=endpoint_name,
                                   ContentType='text/csv',
                                   Body=payload)


result = response['Body'].read().decode('ascii')
print(result)

[0.33649730682373047, 0.33273303508758545, 0.33076968789100647]


In [12]:
import boto3

runtime_client = boto3.client('runtime.sagemaker')


payload = "1,2,3,4,5"
endpoint_name = 'sagemaker-xgboost-2020-08-10-17-27-18-233'

response = runtime_client.invoke_endpoint(EndpointName=endpoint_name,
                                   ContentType='text/csv',
                                   Body=payload)


result = response['Body'].read().decode('ascii')
print(result)

[0.33649730682373047, 0.33273303508758545, 0.33076968789100647]
